In [1]:
# System
import datetime
import traceback
import ast
import os
# import datetime
from collections import Counter
import math
# ML
import numpy as np
import pandas as pd
import pickle
# WEB
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
# NLTK
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By

In [2]:
from sklearn.externals import joblib
filename = "../Models/Models_LR_2019-05-05.joblib"
lr = joblib.load(filename) 

import pickle
filename = "../Frequency_models/word_frequency_2019-05-05.picle"
pickle_in = open(filename,"rb")
words_frequency = pickle.load(pickle_in)

In [3]:
df = pd.read_excel("/home/domantas/Downloads/EM-WR specs for schema (2).xlsx", sheet_name='master list')
df = df[['Country', 'Retailer', 'Robot_id', 'Website']]
df = df[df['Website'].notnull()]
df = df.rename(index=str, columns={"Website": "URL"})

In [151]:
req = urllib.request.Request('https://www.clarins.co.uk/', headers=hdr)
html = urlopen(req, timeout=15).read()
soup = BeautifulSoup(html, "html.parser")
[tag.decompose() for tag in soup("script")]
[tag.decompose() for tag in soup("style")]
text = soup.get_text()
lines = (line.strip() for line in text.splitlines())
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
# Tokenize text
tokens = [token.lower() for token in word_tokenize(text)]
# Remove stopwords
tokens_lemmatize = remove_stopwords(tokens)
en_tokens, en_confidence = get_en_words(tokens_lemmatize)
foreign_words = list(set(tokens_lemmatize) - set(en_tokens))

In [156]:
foreign_words

['',
 '1943',
 'feb',
 'taiwan',
 'paypal',
 '1980',
 'deutsch',
 '1981',
 'email',
 '3',
 '1994',
 '900am',
 '1997',
 '29',
 '28',
 '2014',
 'contouring',
 '17',
 'deutschland',
 'started',
 'cyber',
 '1948',
 'metime',
 '1950',
 '’',
 'makeup',
 'nov',
 '1971',
 '1975',
 '1955',
 'customize',
 '©clarins',
 'nederland',
 'eyeliner',
 '0',
 '900',
 'singapore',
 'français',
 '1998',
 '1985',
 'sunkissed',
 'firming',
 '1958',
 '5',
 '7',
 '01279',
 '1967',
 '2005',
 '530',
 'verfied',
 '2006',
 '1968',
 'uk',
 'malaysia',
 '1966',
 'hydrating',
 '1954',
 'antiaging',
 '1942',
 '1962',
 '1964',
 '1976',
 '16',
 'personalised',
 '1957',
 '9',
 '2011',
 '11',
 'trafficking',
 'rebalancing',
 'jul',
 'moisturizers',
 '£50',
 '2012',
 'korea',
 'highlighter',
 'dec',
 'moisturisers',
 'expired',
 '2000',
 '215',
 'processing',
 '1969',
 'poland',
 'thailand',
 'oct',
 '2017',
 '26',
 '1996',
 '1961',
 'africa',
 '1965',
 'howtos',
 '2010',
 '20',
 '1946',
 '£75',
 'bestseller',
 '22',
 '201

In [79]:
en_tokens, en_confidence = get_en_words(tokens_lemmatize)
foreign_words = list(set(tokens_lemmatize) - set(en_tokens))

In [147]:
def crawl():
    tokens_lemmatize = ''
#     req = urllib.request.Request(row['URL'], headers=hdr)
#     html = urlopen(req, timeout=15).read()
    scrapper.get(row['URL'])
    html = scrapper.page_source
    soup = BeautifulSoup(html, "html.parser")
    [tag.decompose() for tag in soup("script")]
    [tag.decompose() for tag in soup("style")]
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
    # Tokenize text
    tokens = [token.lower() for token in word_tokenize(text)]
    # Remove stopwords
    tokens_lemmatize = remove_stopwords(tokens)
    return tokens_lemmatize if len(tokens_lemmatize) else ''

def remove_stopwords(tokens):
    wnl = WordNetLemmatizer()
    for i, token in enumerate(tokens):
        tokens[i] = ''.join([ch for ch in token if ch not in char_blacklist])
    tokens_sw = [w.lower() for w in tokens if w not in stopwords]
    tokens_lemmatize = [wnl.lemmatize(token) for token in tokens_sw]
    return tokens_lemmatize

def get_en_words(tokens_lemmatize):
    english_tokens = []
    for word in tokens_lemmatize:
        english_tokens.append(word) if word in english_vocab else ''
    english_confidence = round(len(english_tokens) / len(tokens_lemmatize) * 100, 2) if len(english_tokens) > 0 else 0
    return english_tokens, english_confidence

def translate_words():
    foreign_words = list(set(tokens_lemmatize) - set(en_tokens))
    translated_words = []
    if len(foreign_words):
        chunk_size = math.ceil(len(foreign_words) / 5000)
        chunks = np.array_split(foreign_words, chunk_size)
        for chunk in chunks:
            foreign_text = " ".join(chunk)
            input_box = driver.find_element_by_id('source')
            driver.execute_script(f"document.getElementById('source').value = '{foreign_text}';")
            # input_box.send_keys(foreign_text)
            WebDriverWait(driver, 20).until(ec.visibility_of_element_located((By.CSS_SELECTOR, "span.tlid-translation.translation")))
            output_box = driver.find_element_by_css_selector("span.tlid-translation.translation").text
            translated_words.extend(output_box.split(' '))
            input_box.clear()
    return translated_words

In [148]:
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('--ignore-certificate-errors')
options.add_argument('user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"')
# options.add_argument("--test-type")
# driver = webdriver.Chrome('/home/domantas/Documents/selenium_drivers/chromedriver')
driver = webdriver.Chrome('/home/domantas/Documents/selenium_drivers/chromedriver', chrome_options=options)
driver.get("https://translate.google.com/")

scrapper = webdriver.Chrome('/home/domantas/Documents/selenium_drivers/chromedriver', chrome_options=options)
# scrapper.manage().timeouts().pageLoadTimeout(20, TimeUnit.SECONDS);

english_vocab = set(w.lower() for w in nltk.corpus.words.words('en'))
english_tolerance = 50
english_confidence = []

char_blacklist = list(chr(i) for i in range(32, 127) if (i <= 47 or i >= 58)\
                      and (i <= 64 or i >= 91) and (i <= 96 or i >= 123))
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(char_blacklist)
top = 2500
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

df['tokens_en'] = ''
df['en_confidence'] = ''

output_path = '../Datasets/custom_data.csv'

main_counter = 0

for row_id, row in df.iterrows():
    main_counter += 1
    try:   
        tokens = crawl()
        tokens_lemmatize = remove_stopwords(tokens)
        en_tokens, en_confidence = get_en_words(tokens_lemmatize)
        translated_words = translate_words()
        en_tokens_tr, en_confidence_tr = get_en_words(translated_words)
        en_tokens.extend(remove_stopwords(en_tokens_tr))

        df.at[row_id, 'tokens_en'] = en_tokens if len(en_tokens) else ''
        df.at[row_id, 'en_confidence'] = round(len(en_tokens) / len(tokens_lemmatize) * 100, 2) if len(en_tokens) > 0 else 0

        from collections import Counter
        counter = 0
        features_pred = np.zeros(top * len(words_frequency)).reshape(len(words_frequency), top)
        c = [word for word, word_count in Counter(tokens_lemmatize).most_common(top)]
        for category in words_frequency.keys():
            for word in c:
                if word in words_frequency[category]:
                    features_pred[counter][words_frequency[category].index(word)] = 1
            counter+=1

        category_weight = []
        for i in features_pred:
            weight_cof = np.where(i == 1)[0]
            weight_sum = 0
            for cof in weight_cof:
                weight_sum += top - cof
            category_weight.append(weight_sum)

        cat_index = category_weight.index(max(category_weight))
        category = list(words_frequency.keys())[cat_index]
        feature = features_pred[cat_index].reshape(-1, top)
        print(f"url: {row['URL']} | counter: {main_counter} / {df.shape[0]}")
    #     print('Category: ', row['Category'])
        print("My model: ",category)
        lr_prediction = lr.predict(feature)
        print("LR prediction: ", list(words_frequency.keys())[int(lr_prediction[0])])
        df.at[row_id, 'Weight_model'] = category
        df.at[row_id, 'lr_pred'] = list(words_frequency.keys())[int(lr_prediction[0])]        
        
    except Exception:
        print(f"{main_counter}/{df.shape[0]} || FAILED. {row['URL']}")
#         print(traceback.print_exc())
        continue


driver.close()
# df = df[df['tokens_en'] != '']
# df.to_csv(output_path, index=False)

/home/domantas/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys
/home/domantas/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


url: https://www.benefitcosmetics.com/us/en/shop-makeup | counter: 1 / 1815
My model:  Beauty_and_Fitness
LR prediction:  Internet_and_Telecom
url: https://www.benefitcosmetics.com/uk/en-gb/shop-makeup | counter: 2 / 1815
My model:  Beauty_and_Fitness
LR prediction:  Internet_and_Telecom
url: https://us.myprotein.com/ | counter: 3 / 1815
My model:  Beauty_and_Fitness
LR prediction:  Internet_and_Telecom
url: https://www.myprotein.com/ | counter: 4 / 1815
My model:  Beauty_and_Fitness
LR prediction:  Internet_and_Telecom
url: https://www.zara.com/us/ | counter: 5 / 1815
My model:  Shopping
LR prediction:  Recreation_and_Hobbies
url: https://www.zara.com/uk/ | counter: 6 / 1815
My model:  Shopping
LR prediction:  Recreation_and_Hobbies
url: https://www.bershka.com/us/woman-c1010193132.html | counter: 7 / 1815
My model:  Internet_and_Telecom
LR prediction:  Travel
url: https://www.bershka.com/gb/woman-c1010193132.html | counter: 8 / 1815
My model:  Internet_and_Telecom
LR prediction:  Tra

KeyboardInterrupt: 

In [22]:
from openpyxl import Workbook
df.to_excel('VIA_untrained_pred.xlsx', index=False)

In [13]:
# df = df[df['Weight_model'] != '']
model_acc = len(df[df['Weight_model'] == df['Category']]) / len(df) * 100
lr_acc = len(df[df['lr_pred'] == df['Category']]) / len(df) * 100
print("My model accuracy: {}% | {} / {}".format(model_acc, len(df[df['Weight_model'] == df['Category']]), len(df)))
print("LR accuracy: {}% | {} / {}".format(lr_acc, len(df[df['lr_pred'] == df['Category']]), len(df)))

KeyError: 'Category'

In [30]:
df.head()['URL'].tolist()

['https://www.benefitcosmetics.com/us/en/shop-makeup',
 'https://www.benefitcosmetics.com/uk/en-gb/shop-makeup',
 'https://us.myprotein.com/',
 'https://www.myprotein.com/',
 'https://www.zara.com/us/']